<h1>14. Introduction to Pandas</h1>
<h2>11/4/2020</h2>

<h2>14.0 Last Time...</h2>
<ul>
    <li>The <b>operator</b> package enables us to use a function called <b>attrgetter()</b> to grab attribute information from various classes.</li>
    <li>The <b>sorted()</b> function lets us sort data alphabetically or numerically as needed.</li>
    <li>NumPy's <b>meshgrid()</b> module lets us create a grid from lat/lon vectors.</li> 
</ul>

<h2>14.1 Why Pandas?</h2>

Python's standard library has some tools for working with CSV files, but Pandas makes life a whole lot easier! It handles missing data well and also allows for quick calculations and plotting (which we'll be talking about in a couple weeks). A Pandas <b>dataframe</b> is a lot like an Excel spreadsheet, but it's a lot faster and more flexible.

In [ ]:
# Here's the new library we'll want for today:

import pandas as pd

Let's begin by using a tool we're familiar with that should work pretty well for this kind of data: dictionaries!

In [ ]:
my_dict = {
    'name' : ["John", "Diane", "Joe", "Danielle", "Ashley", "Sam"],
    'age' : [20, 55, 35, 40, 21, 35],
    'salary' : [41000, 73000, 68000, 65000, 50000, 55000],
    'designation' : ["VP", "CEO", "CFO", "VP", "VP", "VP"]
}

print(my_dict['age'])

So far so good, but if we want to get the designation for the employee Sam, that can get a little complex. Pandas will make this endeavor easier and more intuitive.

Pandas features two data structures:
<ul>
    <li><b>Series:</b> 1-D labeled arrays that resemble dictionaries</li>
    <li><b>DataFrame:</b> (most common) 2-D like a spreadsheet</li>
</ul>

In [ ]:
employees = pd.DataFrame(my_dict)
print(employees)

Remember from our object-oriented programming lectures: a DataFrame is just an object! We can list its attributes and methods using dir().

In [ ]:
dir(employees)

In [ ]:
employees = pd.DataFrame(my_dict)
print(employees)

Just like Excel spreadsheets, dataframes are made up of <b>rows</b> and <b>columns</b>. Each column will have the same data type (all ints, all floats, all strings, etc.).

We can then set a column (or multiple columns) as an <b>index</b> that we can use as a shorthand.

In [ ]:
employees.set_index('name', inplace=True)
print(employees)

# The inplace argument above means we're replacing
# our 'default' index with our new index.


In [ ]:
print(employees.index)

In [ ]:
print(employees.columns)

We can then use the <b>loc[]</b> function to access a group of rows and columns by a particular label.

In [ ]:
employees.loc['John']

In [ ]:
# Similarly, we can use 'iloc[]' to refer to a particular index.

employees.iloc[3]

It's also possible to create subsets of dataframes based on their values.

In [ ]:
# Let's only look at employees older than 30.
print(employees[employees.age > 30])

In [ ]:
# Let's look at who's a VP and who isn't.
print(employees['designation'] == 'VP')

In [ ]:
# We can also easily set values or add new columns.

# Change John's age to 22.
employees.loc['John','age'] = 22
print(employees)

In [ ]:
# Let's say we now have information about what city
# everyone's in and we want to add that where we have it.

employees['city'] = 'Montreal'
print(employees)

In [ ]:
# Whoops, Diane's moved to Toronto!

employees.loc['Diane','city'] = 'Toronto'
print(employees)

The <b>describe()</b> function is a helpful summary of statistics.

In [ ]:
employees.age.describe()

In [ ]:
# We can also easily do mathematical operations.

print(employees['age'] + 10)

In [ ]:
# Or use things like value counts!

employees.designation.value_counts()

In [ ]:
# Pandas can also be used to create some useful date ranges!

pd.date_range('1/1/2021','1/12/2021')

<h2>14.2 Pandas Examples</h2>

You'll need to re-run these lines of code to clear out all the changes
we made above...

In [ ]:
employees = pd.DataFrame(my_dict)
employees.set_index('name', inplace=True)

<b>1. Promote Ashley to CEO.</b>

<b>2. Using methods you learned in this lecture, find the mean and standard deviation of salaries.</b>

<b>3. A year has passed! Increase everyone's ages by 1.</b>

<h2>14.3 Pandas With Atmospheric Data</h2>

Let's use a real-life example! This is information from Chris McCray at McGill: Daily weather data for Montreal from 1871-2019. Pandas has a <b>read_csv()</b> function that will come in handy here.

In [ ]:
import pandas as pd

mtl_weather = pd.read_csv('http://www.cdmccray.com/python_tutorial/pandas/montreal_daily_weather.csv')

In [ ]:
print(mtl_weather.columns)

Let's set the index to the date of observation.

In [ ]:
mtl_weather.set_index('Date/Time',inplace=True)

<b>Grouping</b> is a useful feature within Pandas that lets us do the following:
<ul>
    <li>Split the data into groups based on some criteria.</li>
    <li>Apply a function to each group independently.</li>
    <li>Combine the results into a data structure.</li>
</ul>

In [ ]:
# As an example, let's count how much temperature data is available for each year.
a = mtl_weather.groupby('Year').count()['Max Temp (°C)']
print(a)

In [ ]:
a.plot()

Okay, so there's a lot of missing data in the 90s! We can actually remove all rows with missing data by using <b>dropna()</b>. Since there's probably a lot of data missing, let's specify that we're only removing the rows that have missing temperature data.

In [ ]:
print(mtl_weather.shape)
mtl_weather.dropna(subset=['Max Temp (°C)'], inplace=True)
print(mtl_weather.shape)

We can also use <b>groupby()</b> to calculate the average temperature for each year!

In [ ]:
mtl_weather.groupby('Year').mean()['Mean Temp (°C)'].plot(figsize=[25,5])

Okay, well, there's obviously something suspicious going on here - it's not that cold, even in Montreal! Note that these are years where a lot of data is missing - it looks like most of that data's from the warm season.

How would you handle this kind of problem?

In [ ]:
yearly_counts = mtl_weather.groupby('Year').count()['Mean Temp (°C)']
yearly_counts.plot()

Let's set our threshold to about 350 days.

In [ ]:
good_years = yearly_counts[yearly_counts>350].index
print(good_years)

We can now subset the dataframe to only include data from our good years using the <b>isin()</b> function.

In [ ]:
mtl_weather = mtl_weather[mtl_weather.Year.isin(good_years)]
mtl_weather.groupby('Year').mean()['Mean Temp (°C)'].plot(figsize=[25,5])

It's also very easy to get extreme values from the dataset. The <b>nsmallest()</b> function will list the smallest values, and the <b>nlargest()</b> function will list the largest.

In [ ]:
print(mtl_weather['Min Temp (°C)'].nsmallest())

In [ ]:
print(mtl_weather['Max Temp (°C)'].nlargest(10))

In [ ]:
mtl_weather.describe()

<h2>14.4 Take-Home Points</h2>
<ul>
    <li><b>Pandas</b> is a useful way of working with CSV data!</li>
    <li>A <b>dataframe</b> is an object that contains rows and columns, much like an Excel spreadsheet.</li>
    <li><b>loc()</b> will let you identify individual rows, columns, or values.</li>
    <li><b>describe()</b> summarizes statistics for a specified section of a dataframe.</li>
    <li><b>read_csv()</b> will read in a CSV file specified by a file location.</li>
    <li><b>groupby()</b> carries out specific operations on groupings within a dataframe.</li>
</ul>